### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.regression = 1 # don't change this

In [ ]:
system.train_set_end = 0.7 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
system.regression_move_threshold = 0.2 # emits a trade order if the predicted move is bigger than this
system.scale_data = 1

### ------------------------------------------------------------------------------------------------------------

### LinearRegression

In [ ]:
# Train LogisticRegression classifier on train data
reg, scaler = train_reg_ensemble(LinearRegression, data, ensemble_size=10)

In [ ]:
# Test once on train data, to ensure nothing is reversed and the regressor learned
*_, trades = rbacktest(reg, scaler, data, skip_train=0, skip_val=1, skip_test=1,)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### XGBRegressor

In [ ]:
# Train XGBClassifier classifier on train data
reg, scaler = train_reg_ensemble(XGBRegressor, data, ensemble_size=10)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### LGBMRegressor

In [ ]:
# Train LGBMClassifier classifier on train data
reg, scaler = train_reg_ensemble(LGBMRegressor, data, ensemble_size=10)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### RandomForestRegressor

In [ ]:
# Train RandomForestClassifier classifier on train data
reg, scaler = train_reg_ensemble(RandomForestRegressor, data, n_jobs=-1, ensemble_size=10)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### ExtraTreesRegressor

In [ ]:
# Train ExtraTreesClassifier classifier on train data
reg, scaler = train_reg_ensemble(ExtraTreesRegressor, data, ensemble_size=10)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### CatBoostRegressor

In [ ]:
# Train CatBoostClassifier classifier on train data
reg, scaler = train_reg_ensemble(CatBoostRegressor, data, verbose=False, ensemble_size=10)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### Ensemble of all

In [ ]:
# Train ensemble on train data
clf, scaler = train_reg_ensemble([LinearRegression, XGBRegressor, LGBMRegressor, RandomForestRegressor, ExtraTreesRegressor, CatBoostRegressor],
                                 data, ensemble_size=1)

In [ ]:
# Test on val data
equity, pf, trades = rbacktest(clf, scaler, data)

In [ ]:
trades.head()